In [17]:
import sys
import time
import random
import math
import itertools
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sn
import networkx as nx

from copy import deepcopy
from typing import Union, List, Tuple, Dict
from pgmpy.readwrite import XMLBIFReader

from BayesNet import *
from BNReasoner import *
from utils import *

# Initialize
---

In [18]:
# BIFXML files
net = './testing/car_ex.BIFXML'

In [19]:
# constructs a BN object
bn = BayesNet()

# Loads the BN from an BIFXML file
bn.load_from_bifxml(net)

# create reasoner
bnr = BNReasoner(bn)

### PERFORMANCE EVALUATION

In [86]:
evi = {
        'FanBelt': 'Broken', 
        'Leak2': 'False',
        'FuelPump': 'Ok'
}
qry = ['Starts']

start = time.time()
MAP = bnr.MAP(query=qry, evidence=evi, ordering_function=minfill_order)
end = time.time()

print(f"MAP done in {round(end - start, 3)} seconds!\n")

print(f"MAP:\n{MAP}\n")

MAP done in 0.4 seconds!

MAP:
{'f_mult(f_mult(SparkPlugs-f_mult(Distributor-f_mult(FuelPump-f_mult(Starts-f_mult(Leak2-f_mult(Starter-f_mult(EngineCranks-f_mult(f_mult(BatteryPower-f_mult(Leak-f_mult(FanBelt-f_mult(Alternator-Charge)_sum(Alternator))_sum(FanBelt))_sum(Leak))_sum(Charge)-f_mult(BatteryAge-BatteryState)_sum(BatteryAge))_sum(BatteryState))_sum(BatteryPower))_sum(Starter))_sum(Leak2))_sum(EngineCranks))_sum(FuelPump))_sum(Distributor))_sum(SparkPlugs))_max(Starts)': p                  0
inst_Starts    False
Name: 0, dtype: object}



In [87]:
data = []

In [96]:
evi = {
        'FanBelt': 'Broken', 
        'Leak2': 'False',
        'FuelPump': 'Ok'
}
qry = ['Starts']

start = time.time()
MAP = bnr.MAP(query=qry, evidence=evi, ordering_function=minfill_order)
end = time.time()

print(f"MAP done in {round(end - start, 3)} seconds!\n")

print(f"MAP:\n{MAP}\n")

data.append(('MAP', 18, 'minfill', qry, evi, end-start))
data

MAP done in 0.372 seconds!

MAP:
{'f_mult(f_mult(SparkPlugs-f_mult(Distributor-f_mult(FuelPump-f_mult(Starts-f_mult(Leak2-f_mult(Starter-f_mult(EngineCranks-f_mult(f_mult(BatteryPower-f_mult(Leak-f_mult(FanBelt-f_mult(Alternator-Charge)_sum(Alternator))_sum(FanBelt))_sum(Leak))_sum(Charge)-f_mult(BatteryAge-BatteryState)_sum(BatteryAge))_sum(BatteryState))_sum(BatteryPower))_sum(Starter))_sum(Leak2))_sum(EngineCranks))_sum(FuelPump))_sum(Distributor))_sum(SparkPlugs))_max(Starts)': p                  0
inst_Starts    False
Name: 0, dtype: object}



[('MAP',
  18,
  'random',
  ['Starts'],
  {'FanBelt': 'Broken', 'Leak2': 'False', 'FuelPump': 'Ok'},
  0.7777614593505859),
 ('MAP',
  18,
  'random',
  ['Starts'],
  {'FanBelt': 'Broken', 'Leak2': 'False', 'FuelPump': 'Ok'},
  0.6108369827270508),
 ('MAP',
  18,
  'random',
  ['Starts'],
  {'FanBelt': 'Broken', 'Leak2': 'False', 'FuelPump': 'Ok'},
  0.6997759342193604),
 ('MAP',
  18,
  'mindeg',
  ['Starts'],
  {'FanBelt': 'Broken', 'Leak2': 'False', 'FuelPump': 'Ok'},
  0.37189316749572754),
 ('MAP',
  18,
  'mindeg',
  ['Starts'],
  {'FanBelt': 'Broken', 'Leak2': 'False', 'FuelPump': 'Ok'},
  0.41589784622192383),
 ('MAP',
  18,
  'mindeg',
  ['Starts'],
  {'FanBelt': 'Broken', 'Leak2': 'False', 'FuelPump': 'Ok'},
  0.40886926651000977),
 ('MAP',
  18,
  'minfill',
  ['Starts'],
  {'FanBelt': 'Broken', 'Leak2': 'False', 'FuelPump': 'Ok'},
  0.4028937816619873),
 ('MAP',
  18,
  'minfill',
  ['Starts'],
  {'FanBelt': 'Broken', 'Leak2': 'False', 'FuelPump': 'Ok'},
  0.39990329742431

In [97]:
df = pd.DataFrame(data, columns=["Method", "Vars", "Ordering", "Query", "Evidence", "Runtime"])
df

,Method,Vars,Ordering,Query,Evidence,Runtime
0,MAP,18,random,[Starts],"{'FanBelt': 'Broken', 'Leak2': 'False', 'FuelP...",0.777761
1,MAP,18,random,[Starts],"{'FanBelt': 'Broken', 'Leak2': 'False', 'FuelP...",0.610837
2,MAP,18,random,[Starts],"{'FanBelt': 'Broken', 'Leak2': 'False', 'FuelP...",0.699776
3,MAP,18,mindeg,[Starts],"{'FanBelt': 'Broken', 'Leak2': 'False', 'FuelP...",0.371893
4,MAP,18,mindeg,[Starts],"{'FanBelt': 'Broken', 'Leak2': 'False', 'FuelP...",0.415898
5,MAP,18,mindeg,[Starts],"{'FanBelt': 'Broken', 'Leak2': 'False', 'FuelP...",0.408869
6,MAP,18,minfill,[Starts],"{'FanBelt': 'Broken', 'Leak2': 'False', 'FuelP...",0.402894
7,MAP,18,minfill,[Starts],"{'FanBelt': 'Broken', 'Leak2': 'False', 'FuelP...",0.399903
8,MAP,18,minfill,[Starts],"{'FanBelt': 'Broken', 'Leak2': 'False', 'FuelP...",0.371894


In [98]:
df.to_csv("outputs/map_18_q3.csv", sep='\t', index=False)

# MPE

In [115]:
evi = {'Starts': 'Yes'}

start = time.time()
MPE = bnr.MPE(evidence=evi, ordering_function=random_order)
end = time.time()

print(f"MPE done in {round(end - start, 3)} seconds!\n")

print(f"MPE:\n{[MPE[key] for key in MPE]}")

MPE done in 0.831 seconds!

MPE:
[p                        0
inst_EngineCranks    False
inst_Radio           False
Name: 0, dtype: object]


In [147]:
data = []

In [150]:
evi = {'EngineCranks': 'Cranks'}

start = time.time()
MPE = bnr.MPE(evidence=evi, ordering_function=random_order)
end = time.time()

print(f"MPE done in {round(end - start, 3)} seconds!\n")

print(f"MPE:\n{MPE}")

data.append(('MPE', 18, 'random', evi, end-start))
data

MPE done in 0.576 seconds!

MPE:
{'f_mult(f_mult(GasInTank-f_mult(GasGauge)_max(GasGauge))_max(GasInTank)-f_mult(Lights)_max(Lights)-f_mult(Radio)_max(Radio)-f_mult(Distributor-f_mult(Leak2-f_mult(FuelPump-f_mult(f_mult(Starter-EngineCranks)_max(Starter)-f_mult(f_mult(SparkPlugs-Starts)_max(SparkPlugs))_max(Starts))_max(EngineCranks))_max(FuelPump))_max(Leak2))_max(Distributor)-f_mult(f_mult(BatteryAge-BatteryState)_max(BatteryAge)-f_mult(FanBelt-f_mult(BatteryPower-f_mult(Leak-f_mult(Alternator-Charge)_max(Alternator))_max(Leak))_max(Charge))_max(FanBelt))_max(BatteryState))_max(BatteryPower)': inst_GasInTank       False
inst_Lights          False
inst_Radio           False
inst_Distributor     False
p                        0
inst_BatteryState    False
inst_BatteryPower    False
Name: 0, dtype: object}


[('MPE', 18, 'random', {'EngineCranks': 'Cranks'}, 0.7677385807037354),
 ('MPE', 18, 'random', {'EngineCranks': 'Cranks'}, 0.6298301219940186),
 ('MPE', 18, 'random', {'EngineCranks': 'Cranks'}, 0.5758273601531982)]

In [144]:
df = pd.DataFrame(data, columns=["Method", "Vars", "Ordering", "Evidence", "Runtime"])
df

,Method,Vars,Ordering,Evidence,Runtime
0,MPE,18,random,{'Starts': 'No'},1.795438
1,MPE,18,random,{'Starts': 'No'},2.678156
2,MPE,18,random,{'Starts': 'No'},1.335584
3,MPE,18,mindeg,{'Starts': 'No'},0.455864
4,MPE,18,mindeg,{'Starts': 'No'},0.470877
5,MPE,18,mindeg,{'Starts': 'No'},0.511849
6,MPE,18,minfill,{'Starts': 'No'},0.469880
7,MPE,18,minfill,{'Starts': 'No'},0.475869
8,MPE,18,minfill,{'Starts': 'No'},0.483856


In [146]:
# df.to_csv("outputs/mpe_18_q2.csv", sep='\t', index=False)

# Usage
---

In [ ]:
# query inputs
# evidence = {'hear-bark': True, 'family-out': True}
# x = ['light-on']
# prio = ['dog-out']

### D-separation
Checks whether x and y are d-separated, given (possibly empty) evidence. 

* x and y must be lists
* evidence is (always) a dict
* output is boolean

In [ ]:
# dsep = bnr.d_separated(prio, x, evidence)
# print(f"Are {prio} and {x} d-separated by evidence '{evidence.keys()} = {evidence.values()}'?\n\n{dsep}")

### Ordering functions for determining elimination order
inputs should be network structure
* y = list of query variables
* z = priority variables which are ordered first __[optional]__

In [ ]:
# pi1 = mindeg_order(net=bn, query=prio)
# pi2 = minfill_order(net=bn, query=prio)
# print(f"Variable orderings mindeg:\t{pi1}")
# print(f"Variable orderings minfill:\t{pi2}")

### Marginal distributions
with optional ordering function (name of function is input)

In [ ]:
# without evidence:
# prior_marginal = bnr.compute_marginal(query=x, ordering_function=random_order)
# print(f"\n\nPrior Marginal for Query {x}\n{prior_marginal}")

# with evidence:
# posterior_marginal = bnr.compute_marginal(query=x, evidence=evidence, ordering_function=minfill_order)
# print(f"\nPosterior Marginal for Query {x} with Evidence {evidence}\n{posterior_marginal}")

### Most likely instantiations

In [ ]:
# MAP:
# MAP = bnr.MAP(query=x, evidence=evidence)
# print(f"MAP:\n{[MAP[key] for key in MAP]}\n")

# MPE:
# MPE = bnr.MPE(evidence=evidence, ordering_function=minfill_order)
# print(f"MPE:\n{[MPE[key] for key in MPE]}")